In [1]:
import websocket
import json
import ccxt
import time

from itertools import permutations
from concurrent.futures import ThreadPoolExecutor
from binance.client import Client

def get_binance_symbols():
    binance = ccxt.binance()
    markets = binance.load_markets()
    all_symbols = set()
    for k in markets.keys():
        all_symbols.add(markets[k]['id'])
    return all_symbols

def my_cry_symbols(cryptos = ['BTC','ETH','USDT','BNB','XRP','ADA','SOL','LINK','EUR']):

    # Generate all possible permutations without repetitions
    all_permutations = list(permutations(cryptos, 2))  # Change 2 to the desired permutation length

    # Print all filtered permutations
    return set([''.join(p) for p in all_permutations])

def cry_symbols():
    return list(get_binance_symbols().intersection(my_cry_symbols()))

def on_message(symbol, message, ws):
    data = json.loads(message)
    kline = data['k']

    if kline['x']:
        print(f'''
            Symbol: {symbol},
            Time: {kline['t']},
            Open: {kline['o']},
            High: {kline['h']},
            Low: {kline['l']},
            Close: {kline['c']},
            kline_closed: {kline['x']}
            '''
        )

        # Close the WebSocket connection after processing the first kline
        ws.close()

# Create a WebSocket connection for each symbol
def subscribe_to_symbol(symbol):
    ws = websocket.WebSocketApp(f'wss://stream.binance.com:9443/ws/{symbol.lower()}@kline_1m',
                                on_message=lambda ws,
                                msg: on_message(symbol, msg, ws)
                                )
    ws.run_forever()

# Use ThreadPoolExecutor to run WebSocket connections in parallel
def main():
    # Binance API credentials
    api_key = open("API.txt", "r").read()
    api_secret = open('KY.txt', 'r').read()

    symbols = cry_symbols()

    # Binance WebSocket URLs for multiple trading pairs
    urls = [f'wss://stream.binance.com:9443/ws/{symbol.lower()}@kline_1m' for symbol in symbols]

    # Binance client setup
    binance_client = ccxt.binance({
        'apiKey': api_key,
        'secret': api_secret,
    })

    with ThreadPoolExecutor(max_workers=len(symbols)) as executor:
        executor.map(subscribe_to_symbol, symbols)

if __name__ == "__main__":
    main()



            Symbol: BNBEUR,
            Time: 1702220640000,
            Open: 223.60000000,
            High: 223.80000000,
            Low: 223.60000000,
            Close: 223.80000000,
            kline_closed: True
            
            Symbol: SOLETH,
            Time: 1702220640000,
            Open: 0.03139000,
            High: 0.03141000,
            Low: 0.03139000,
            Close: 0.03141000,
            kline_closed: True
            

            Symbol: BNBETH,
            Time: 1702220640000,
            Open: 0.10240000,
            High: 0.10250000,
            Low: 0.10240000,
            Close: 0.10250000,
            kline_closed: True
            

            Symbol: LINKEUR,
            Time: 1702220640000,
            Open: 14.94700000,
            High: 14.94700000,
            Low: 14.94700000,
            Close: 14.94700000,
            kline_closed: True
            

            Symbol: XRPEUR,
            Time: 1702220640000,
            Open: 0.61